# RAPIDS cuML TfidfVectorizer and KNN to find similar Text and Images
In this notebook we use RAPIDS cuML's TfidfVectorizer and cuML's KNN to find items with similar titles and items with similar images. First we use RAPIDS cuML TfidfVectorizer to extract text embeddings of each item's title and then compare the embeddings using RAPIDS cuML KNN. Next we extract image embeddings of each item with EffNetB0 and compare them using RAPIDS cuML KNN.[](http://)

## Load Libraries

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2, matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
print('TF',tf.__version__)

TF 2.4.1


In [7]:
# RESTRICT TENSORFLOW TO 12GB OF GPU RAM
# SO THAT WE HAVE GPU RAM FOR RAPIDS CUML KNN
LIMIT = 12
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*LIMIT)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)
print('Restrict TensorFlow to max %iGB GPU RAM'%LIMIT)
print('so RAPIDS can use %iGB GPU RAM'%(16-LIMIT))

1 Physical GPUs, 1 Logical GPUs
Restrict TensorFlow to max 12GB GPU RAM
so RAPIDS can use 4GB GPU RAM


## Load Train Data
In this competition, we have items with an image and title. For the train data, the column `label_group` indicates the ground truth of which items are similar. We need to build a model that finds these similar images based on their image and title's text. In this notebook we explore some tools to help us.

In [ ]:
train_df = pd.read_csv('../input/shopee-product-matching/train.csv')
print('train shape is', train_df.shape )
train_df.head()

In [ ]:
# let's see the number of unique label groups we have
train_df['label_group'].nunique() 
print("Number of unique groups are {} out of total records of {}".format(train_df['label_group'].nunique(), len(train_df)))

This means on an average a group would contain 3-4 items

In [ ]:
train_df.info()

Dataset is clean, no null values in the dataset

## Displaying Random Images from Training Dataset 

In [ ]:
BASE = '../input/shopee-product-matching/train_images/'

def display_image(train_df, rows, cols, path=BASE):
    
    for i in range(rows):
        plt.figure(figsize=(20,5))
        for j in range(cols):
            row = np.random.randint(0, len(train_df)) # picking a random row
            image_hash = train_df.iloc[row, 1]
            image_decoded = cv2.imread(path + image_hash)
            title = train_df.iloc[row, 3]
            
            text_ordering = ""
            for i,ch in enumerate(title):
                text_ordering += ch
                if (i!=0) & (i%20 == 0):
                    text_ordering += '\n'
                    
            ## plot it out!
            plt.subplot(1,cols,j+1) # subplot takes (rows, columns, nth plot)
            plt.title(text_ordering)
            plt.axis("off")
            plt.imshow(image_decoded)
            
    
    plt.show()


display_image(train_df,4,6)

## Displaying some Group items

In [ ]:
groups = train_df.label_group.value_counts()
plt.figure(figsize=(20,5))
plt.bar(groups.index.values[:50].astype(str), groups.values[:50])
plt.xticks(rotation = 50)
plt.xlabel("Group Label")
plt.ylabel("Count of number of Items in the group")
plt.title("Top 50 Groups according to the frequency of the number of items in them")
plt.show()

In [ ]:
for k in range(5):
    print("#"*40)
    print("### Group {} with label: {}".format(k+1, groups.index.values[k].astype(str)))
    print("#"*40)
    Kth_df = train_df.loc[train_df.label_group == groups.index[k]]
    display_image(Kth_df, 2, 4)
    print("\n\n")

# Finding Similar Titles

In [2]:
!pip install cuml

In [4]:
import cuml, cudf, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors
print("RAPIDS CuML version: ", cuml.__version__)

RAPIDS CuML version:  0.16.0


In [22]:
# LOAD Training Data into GPU using CUDF!!!
train_df_gpu = cudf.read_csv("../input/shopee-product-matching/train.csv")
print("Shape of dataset: ", train_df_gpu.shape)
train_df_gpu.head()

Shape of dataset:  (34250, 5)


,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069


### Extracting Text Embeddings using TfidfVectorizor

In [23]:
model = TfidfVectorizer(stop_words = 'english', binary=True)
text_embeddings = model.fit_transform(train_df_gpu.title).toarray()
print("Shape of text embeddings: ", text_embeddings.shape)

Shape of text embeddings:  (34250, 24939)


In [24]:
text_embeddings ## Sparse Matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

### Classifying Similar Titles using KNN

In [25]:
knn_model = NearestNeighbors(n_neighbors=50)
knn_model.fit(text_embeddings)
distances, indices = knn_model.kneighbors(text_embeddings)

In [27]:
print("Distances: ", distances, "\n")
print("Indices: ", indices)

Distances:  [[0.         0.         1.0958833  ... 1.2939385  1.2949737  1.2954024 ]
 [0.         0.9047     0.9397362  ... 1.2889568  1.2915127  1.2919577 ]
 [0.         0.8221763  1.280804   ... 1.3698959  1.3700792  1.3700906 ]
 ...
 [0.         0.8017645  0.846956   ... 1.2954924  1.2956679  1.2960013 ]
 [0.         0.81143415 0.82256556 ... 1.327445   1.3280029  1.328192  ]
 [0.         0.8836633  1.1210921  ... 1.2967644  1.2970171  1.3020754 ]] 

Indices:  [[    0 33161  9219 ... 14456  7147  7433]
 [    1 18683 23593 ... 21923   666 12203]
 [    2 15668  6353 ... 21075 11305  7202]
 ...
 [34247 18379 13116 ... 33794 10320 29336]
 [34248 13887 20813 ... 22306  5554 14443]
 [34249 33792 13152 ... 17742 30214 15478]]


Gives the indices of similar titles

In [29]:
## Printing first 10 items of each of the 5 groups of titles obtained by tuning the KNN model on text embeddings

for k in range(5):
    print(train_df_gpu.loc[cupy.asnumpy(indices[k,:10]), ['title', 'label_group']])
    print("\n")
    print("\n")

                                                   title  label_group
0                              Paper Bag Victoria Secret    249114794
33161                          PAPER BAG VICTORIA SECRET    249114794
9219                             PAPER BAG THE BODY SHOP    471581622
25386  READY! GRATIS ISI ULANG! Pengharum parfum mobi...    866425052
31159                      Paper Bag The Body Shop Small    471581622
8469   b"victoria iria Goat's Milk Hand Body Lotion 2...    865318054
2676           Paper Bag Tas Kertas Polos Ukuran 18x5x22    982607408
30518                           VELROSE SECRET FACE MASK   2992234715
4156   Dengan Paper Bag TABITA EXCLUSIVE ORIGINAL KEM...   2591192196
28498              WARDAH WHITE SECRET FACIAL WASH 100ml   1952691729




                                                   title  label_group
1      Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...   2937985045
18683  Double Tape VHB 3M ORIGINAL 12mm x 4.5mm Busa ...   2937985045
23593  Double Ta

# Finding Similar Images

### Extracting Image Embeddings using EffNetB0

In [31]:
BASE = "../input/shopee-product-matching/train.csv"

class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, df, img_size=256, batch_size=32, path=BASE): 
        self.df = df
        self.img_size = img_size
        self.batch_size = batch_size
        self.path = path
        self.indexes = np.arange( len(self.df) )
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        ct = len(self.df) // self.batch_size
        ct += int(( (len(self.df)) % self.batch_size)!=0)
        return ct

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X = self.__data_generation(indexes)
        return X
            
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' 
        X = np.zeros((len(indexes),self.img_size,self.img_size,3),dtype='float32')
        df = self.df.iloc[indexes]
        for i,(index,row) in enumerate(df.iterrows()):
            img = cv2.imread(self.path+row.image)
            X[i,] = cv2.resize(img,(self.img_size,self.img_size)) #/128.0 - 1.0
        return X

In [ ]:
model = EfficientNetB0(weights='imagenet')